# 아파트 실거래가 예측 Competition

In [111]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Data Import

In [112]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

In [113]:
train.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [114]:
test.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,부산광역시,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


In [115]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216553 entries, 0 to 1216552
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   transaction_id          1216553 non-null  int64  
 1   apartment_id            1216553 non-null  int64  
 2   city                    1216553 non-null  object 
 3   dong                    1216553 non-null  object 
 4   jibun                   1216553 non-null  object 
 5   apt                     1216553 non-null  object 
 6   addr_kr                 1216553 non-null  object 
 7   exclusive_use_area      1216553 non-null  float64
 8   year_of_completion      1216553 non-null  int64  
 9   transaction_year_month  1216553 non-null  int64  
 10  transaction_date        1216553 non-null  object 
 11  floor                   1216553 non-null  int64  
 12  transaction_real_price  1216553 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 120.7+ M

In [116]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          5463 non-null   int64  
 1   apartment_id            5463 non-null   int64  
 2   city                    5463 non-null   object 
 3   dong                    5463 non-null   object 
 4   jibun                   5463 non-null   object 
 5   apt                     5463 non-null   object 
 6   addr_kr                 5463 non-null   object 
 7   exclusive_use_area      5463 non-null   float64
 8   year_of_completion      5463 non-null   int64  
 9   transaction_year_month  5463 non-null   int64  
 10  transaction_date        5463 non-null   object 
 11  floor                   5463 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 512.3+ KB


***
# Making Features

In [117]:
train['city_dong'] = train.city + train.dong
test['city_dong'] = test.city + test.dong

In [118]:
tr_n_apt = train.groupby('city_dong').apt.nunique().reset_index(name = 'n_apt')

In [119]:
train = pd.merge(train, tr_n_apt, how = 'left', on = 'city_dong')
test = pd.merge(test, tr_n_apt, how = 'left', on = 'city_dong')

In [120]:
tr_skew_cnt = train.groupby('city_dong').transaction_year_month.skew().reset_index(name = 'skew_cnt')

In [121]:
train = pd.merge(train, tr_skew_cnt, how = 'left', on = 'city_dong')
test = pd.merge(test, tr_skew_cnt, how = 'left', on = 'city_dong')

In [122]:
tr_skew_year = train.groupby('city_dong').year_of_completion.skew().reset_index(name = 'skew_year')

In [123]:
train = pd.merge(train, tr_skew_year, how = 'left', on = 'city_dong')
test = pd.merge(test, tr_skew_year, how = 'left', on = 'city_dong')

In [124]:
y = np.log1p(train.transaction_real_price)

In [125]:
kf = KFold(n_splits = 20, random_state = 524, shuffle = True)

In [126]:
def get_date(x) :
    if x == '1-10' :
        return '월초'
    elif x == '11-20' :
        return '중순'
    else :
        return '월말'

In [127]:
train['transaction_date'] = train['transaction_date'].apply(get_date)
test['transaction_date'] = test['transaction_date'].apply(get_date)

In [128]:
X.skew()

skew_cnt                 -0.701117
n_apt                    -0.410704
exclusive_use_area       -0.438156
year_of_completion       -0.470423
transaction_year_month   -0.267548
floor                     1.324710
city_부산광역시                0.451716
city_서울특별시               -0.451716
transaction_date_월말       0.000000
dtype: float64

In [129]:
np.log1p(X).skew()

skew_cnt                 -1.827812
n_apt                    -1.296958
exclusive_use_area       -0.862525
year_of_completion       -0.481260
transaction_year_month   -0.268997
floor                          NaN
city_부산광역시                     NaN
city_서울특별시                     NaN
transaction_date_월말            NaN
dtype: float64

# Modeling

In [131]:
X = train[['skew_year', 'skew_cnt', 'n_apt', 'city', 'exclusive_use_area', 'year_of_completion', 'transaction_year_month', 'transaction_date', 'floor']]

In [132]:
X = pd.get_dummies(columns = ['city', 'transaction_date'], data = X)

In [133]:
target = test[['skew_year', 'skew_cnt', 'n_apt', 'city', 'exclusive_use_area', 'year_of_completion', 'transaction_year_month', 'transaction_date', 'floor']]

In [134]:
target = pd.get_dummies(columns = ['city', 'transaction_date'], data = target)

In [135]:
X[['n_apt', 'exclusive_use_area']] = np.log1p(X[['n_apt', 'exclusive_use_area']])
target[['n_apt', 'exclusive_use_area']] = np.log1p(target[['n_apt', 'exclusive_use_area']])

## LGBMRegressor

In [136]:
lgbm = LGBMRegressor(random_state = 524, max_depth = 5, n_estimators = 10000, learning_rate = 0.08, objective = 'rmse')

In [137]:
rmse_list = []
lgbm_pred = np.zeros((target.shape[0]))
i = 0
for tr_idx, val_idx in kf.split(X, y) :
    i += 1
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 5000, early_stopping_rounds = 1000)
    pred = [0 if x <0 else x for x in lgbm.predict(val_x)]
    sub_pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(target)]) / 20
    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), np.expm1(pred)))
    print(f'{i}FOLD Training....val_RMSE : {rmse}\n')
    rmse_list.append(rmse)
    lgbm_pred += sub_pred
print(f'\n{lgbm.__class__.__name__}의 20FOLD 평균 RMSE는 {np.mean(rmse_list)}')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 1000 rounds
[5000]	training's rmse: 0.102197	valid_1's rmse: 0.105048
[10000]	training's rmse: 0.0905574	valid_1's rmse: 0.0948086
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.0905574	valid_1's rmse: 0.0948086
1FOLD Training....val_RMSE : 4740.187986888123

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 1000 rounds
[5000]	training's rmse: 0.101874	valid_1's rmse: 0.103042
[10000]	training's rmse: 0.0904679	valid_1's rmse: 0.0932242
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.0904679	valid_1's rmse: 0.0932242
2FOLD Training....val_RMSE : 4935.496719572434

[LightGBM] [Warning] Accuracy may be bad since you didn't expl

[10000]	training's rmse: 0.0904838	valid_1's rmse: 0.0965104
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.0904838	valid_1's rmse: 0.0965104
18FOLD Training....val_RMSE : 4620.65682681911

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 1000 rounds
[5000]	training's rmse: 0.101944	valid_1's rmse: 0.107404
[10000]	training's rmse: 0.090158	valid_1's rmse: 0.097458
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.090158	valid_1's rmse: 0.097458
19FOLD Training....val_RMSE : 4875.936277536051

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 1000 rounds
[5000]	training's rmse: 0.102027	valid_1's rmse: 0.103405
[10000]	training's rmse: 0.0903892	valid_1's rmse: 0.0932646
Did 

# Submission

In [138]:
submission['transaction_real_price'] = lgbm_pred# * .5 + xgb_pred  * .25 + cb_pred * .25

In [139]:
submission.to_csv('0523.csv', index = False)

In [141]:
submission

,transaction_id,transaction_real_price
0,1145756,26056.509497
1,1198704,13060.081991
2,1222384,11495.061428
3,1179897,111142.559723
4,1223091,46945.772260
...,...,...
5458,1174640,70185.852771
5459,1175575,196419.467890
5460,1157024,63713.187925
5461,1136863,33197.524333


lgbm = 4388.8256445511